In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
job_record = pd.read_csv('../data/fullsample.csv',
                        parse_dates = ['BEGIN',
                                      'END'])

job_record.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0


In [9]:
job_record['BEGIN'] = pd.to_datetime(job_record['BEGIN'],
                                     infer_datetime_format = True,
                                    errors = 'coerce')

In [11]:
job_record['END'] = pd.to_datetime(job_record['END'],
                                     infer_datetime_format = True,
                                    errors = 'coerce')

In [15]:
job_record.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7395885 entries, 0 to 7395884
Data columns (total 12 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   JOBID      7395885 non-null  object        
 1   STATE      7395885 non-null  object        
 2   BEGIN      7395442 non-null  datetime64[ns]
 3   END        7395234 non-null  datetime64[ns]
 4   REQMEM     7395885 non-null  object        
 5   USEDMEM    7395885 non-null  object        
 6   REQTIME    7395885 non-null  object        
 7   USEDTIME   7395885 non-null  object        
 8   NODES      7395885 non-null  int64         
 9   CPUS       7395885 non-null  int64         
 10  PARTITION  7395885 non-null  object        
 11  EXITCODE   7395885 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(8)
memory usage: 677.1+ MB


In [16]:
job_record['PARTITION'].value_counts()

production              7019578
nogpfs                   147229
pascal                   124453
sam                       64967
turing                    21424
maxwell                   11278
cgw-maizie                 4309
debug                      1616
cgw-platypus                379
cgw-dsi-gw                  228
cgw-capra1                  157
cgw-dougherty1              125
cgw-horus                    61
cgw-cqs1                     28
cgw-hanuman                  21
cgw-sideshowbob              14
cgw-vm-qa-flatearth1          9
cgw-tbi01                     8
cgw-rocksteady                1
Name: PARTITION, dtype: int64

In [17]:
job_record['EXITCODE'].value_counts()

0:0      7384480
1:0         4958
0:15        1887
0:125       1739
0:9         1361
2:0          508
0:7          389
121:0         89
127:0         88
13:0          68
24:0          67
0:11          35
38:0          32
28:0          29
6:0           27
126:0         24
0:6           18
0:2           16
7:0           12
29:0          12
16:0           9
59:0           8
9:0            4
0:40           4
8:0            3
125:0          3
0:105          2
76:0           1
85:0           1
0:12           1
30:0           1
0:98           1
43:0           1
3:0            1
67:0           1
4:0            1
11:0           1
0:8            1
103:0          1
116:0          1
Name: EXITCODE, dtype: int64